## Jupiter Dependency Check
 Esse notebook irá pegar as tabelas dos sites que contêm a grade curricular do BCC, e as relacionar com seus oferecimentos nos últimos 4 anos. Além disso, também irá verificar se os requisitos de cada disciplina não estão sendo oferecidos em semestres posteriores às que dependem destes.

## Declaração e importação de libs

In [1]:
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from unicodedata import normalize
from webdriver_manager.chrome import ChromeDriverManager

#driver = webdriver.Chrome(
   # executable_path=r'C:/Program Files/webdrivers/chromedriver.exe')

# Pegando a grade curricular e os oferecimentos nos últimos 4 anos.

In [2]:

gradeCurricular = pd.read_html('https://uspdigital.usp.br/jupiterweb/listarGradeCurricular?codcg=45&codcur=45052&codhab=1&tipo=N',
                        match= "Disciplinas");

# [0] = 1° ano; [1] = 2° ano; [3] = 3° e 4° anos

#segundoSemestre2021 = [0]*3;
#primeiroSemestre2021 = [0]*3;
#segundoSemestre2020 = [0]*3;
#primeiroSemestre2020 = [0]*3;
#segundoSemestre2019 = [0]*3;
#primeiroSemestre2019 = [0]*3;


#driver.get('https://www.ime.usp.br/~pmiranda/horarios/2s21/index.html')
#segundoSemestre2021[0]=pd.read_html(driver.page_source)[0]
#segundoSemestre2021[1]=pd.read_html(driver.page_source)[1]
#segundoSemestre2021[2]=pd.read_html(driver.page_source)[2]
#driver.quit

#driver.get('https://www.ime.usp.br/~pmiranda/horarios/1s21/index.html')
#primeiroSemestre2021[0]=pd.read_html(driver.page_source)[0]
#primeiroSemestre2021[1]=pd.read_html(driver.page_source)[1]
#primeiroSemestre2021[2]=pd.read_html(driver.page_source)[2]
#driver.quit

#driver.get('https://www.ime.usp.br/~pmiranda/horarios/2s20/index.html')
#segundoSemestre2020[0]=pd.read_html(driver.page_source)[0]
#segundoSemestre2020[1]=pd.read_html(driver.page_source)[1]
#segundoSemestre2020[2]=pd.read_html(driver.page_source)[2]
#driver.quit

#driver.get('https://www.ime.usp.br/~pmiranda/horarios/1s20/index.html')
#primeiroSemestre2020[0]=pd.read_html(driver.page_source)[0]
#primeiroSemestre2020[1]=pd.read_html(driver.page_source)[1]
#primeiroSemestre2020[2]=pd.read_html(driver.page_source)[2]
#driver.quit

#driver.get('https://www.ime.usp.br/~pmiranda/horarios/2s19/index.html')
#segundoSemestre2019[0]=pd.read_html(driver.page_source)[0]
#segundoSemestre2019[1]=pd.read_html(driver.page_source)[1]
#segundoSemestre2019[2]=pd.read_html(driver.page_source)[2]
#driver.quit

#driver.get('https://www.ime.usp.br/~pmiranda/horarios/1s19/index.html')
#primeiroSemestre2019[0]=pd.read_html(driver.page_source)[0]
#primeiroSemestre2019[1]=pd.read_html(driver.page_source)[1]
#primeiroSemestre2019[2]=pd.read_html(driver.page_source)[2]
#driver.quit

## Pegando os códigos das disciplinas sendo oferecidas:

In [3]:
df = gradeCurricular[3];
#print(s2a21[0])

In [4]:
codigos = {};
count = 0;
for i in df[0]:
    if(isinstance(i, str)):
        if(re.match("^(([A-Z]{3})[0-9]{4})$", i)):
            codigos[i] = []; 
    count +=1;
    # Ao printar os valores de i nota-se que existem valores repetidos após 262,
    # provavelmente um erro de leftovers do jupiter;
    if(count==262):
        break;

## Pegando os requisitos de cada disciplina:

In [7]:
for cod in codigos:   
    requisitosDisciplina = pd.read_html("https://uspdigital.usp.br/jupiterweb/listarCursosRequisitos?coddis="+cod);
    if(len(requisitosDisciplina) >= 6):
        req = requisitosDisciplina[6];
        for i in req:
            print("Req[i]: ", req[i]);
            print("req[i][0]: ", req[i][0]);
            print(len(req[i]))
            j = 1;
            k = 0;
            try:
                while(((re.match("(^(Curso:\ 45051))(.*)", req[i][k]) == None) and
                        (re.match("(^(Curso:\ 45052))(.*)", req[i][k]) == None)) and len(req[i])> k):
                    print("entrou")
                    k+=1;
                    while(isinstance(req[i][k], str) == False): k+=1;
                if(re.match("(^(Curso:\ 45051))(.*)", req[i][k]) or
                   (re.match("(^(Curso:\ 45052))(.*)", req[i][k]))):
                    while(isinstance(req[i][k+j], str)):
                        repetida = 0;
                        if(re.match("(^(([A-Z]{3})[0-9]{4}))(.*)", req[i][k+j])):
                            for z in codigos[cod]:
                                if z == req[i][k+j].split(' ')[0]:
                                    repetida = 1;
                                    break;
                            if repetida == 0:    
                                codigos[cod].append(req[i][k+j].split(' ')[0]);
                        j+=1;
            except:
                print("oops");


Req[i]:  0    Curso: 45051 Bacharelado em Ciência da Computa...
1                    MAC0110 - Introdução à Computação
2                                                  NaN
3    Curso: 45052 Bacharelado em Ciência da Computa...
4                    MAC0110 - Introdução à Computação
5                                                  NaN
Name: 0, dtype: object
req[i][0]:  Curso: 45051 Bacharelado em Ciência da Computação (diurno) - Período ideal: 2
6
Req[i]:  0    Curso: 45051 Bacharelado em Ciência da Computa...
1                    MAC0110 - Introdução à Computação
2                                                  NaN
3    Curso: 45052 Bacharelado em Ciência da Computa...
4                    MAC0110 - Introdução à Computação
5                                                  NaN
Name: 1, dtype: object
req[i][0]:  Curso: 45051 Bacharelado em Ciência da Computação (diurno) - Período ideal: 2
6
Req[i]:  0    Curso: 45051 Bacharelado em Ciência da Computa...
1                           

Req[i]:  0     Curso: 3011 Engenharia - Habilitação: Engenhar...
1            MAT2453 - Cálculo Diferencial e Integral I
2                                                    ou
3            MAT2453 - Cálculo Diferencial e Integral I
4                                                   NaN
5     Curso: 3022 Engenharia - Habilitação: Ciclo Bá...
6            MAT2453 - Cálculo Diferencial e Integral I
7                                                   NaN
8     Curso: 3022 Engenharia - Habilitação: Habilita...
9            MAT2453 - Cálculo Diferencial e Integral I
10                                                  NaN
11    Curso: 3032 Engenharia - Habilitação: Ciclo Bá...
12           MAT2453 - Cálculo Diferencial e Integral I
13                                                  NaN
14    Curso: 3044 Engenharia - Habilitação: Habilita...
15           MAT2453 - Cálculo Diferencial e Integral I
16                                                  NaN
17    Curso: 3052 Engenharia - Habilita

Req[i]:  0    Curso: 45052 Bacharelado em Ciência da Computa...
1         MAC0121 - Algoritmos e Estruturas de Dados I
2                                                  NaN
Name: 0, dtype: object
req[i][0]:  Curso: 45052 Bacharelado em Ciência da Computação (diurno) - Período ideal: 3
3
Req[i]:  0    Curso: 45052 Bacharelado em Ciência da Computa...
1         MAC0121 - Algoritmos e Estruturas de Dados I
2                                                  NaN
Name: 1, dtype: object
req[i][0]:  Curso: 45052 Bacharelado em Ciência da Computação (diurno) - Período ideal: 3
3
Req[i]:  0    Curso: 45052 Bacharelado em Ciência da Computa...
1                                            Requisito
2                                                  NaN
Name: 2, dtype: object
req[i][0]:  Curso: 45052 Bacharelado em Ciência da Computação (diurno) - Período ideal: 3
3
Req[i]:  0    Curso: 45052 Bacharelado em Ciência da Computa...
1                                                  NaN
2             

Req[i]:  0    Curso: 45051 Bacharelado em Ciência da Computa...
1         MAC0121 - Algoritmos e Estruturas de Dados I
2                                                  NaN
3    Curso: 45052 Bacharelado em Ciência da Computa...
4         MAC0121 - Algoritmos e Estruturas de Dados I
5                                                  NaN
Name: 0, dtype: object
req[i][0]:  Curso: 45051 Bacharelado em Ciência da Computação (diurno) - Período ideal: 5
6
Req[i]:  0    Curso: 45051 Bacharelado em Ciência da Computa...
1         MAC0121 - Algoritmos e Estruturas de Dados I
2                                                  NaN
3    Curso: 45052 Bacharelado em Ciência da Computa...
4         MAC0121 - Algoritmos e Estruturas de Dados I
5                                                  NaN
Name: 1, dtype: object
req[i][0]:  Curso: 45051 Bacharelado em Ciência da Computação (diurno) - Período ideal: 5
6
Req[i]:  0    Curso: 45051 Bacharelado em Ciência da Computa...
1                           

Req[i]:  0    Curso: 45052 Bacharelado em Ciência da Computa...
1                      MAC0338 - Análise de Algoritmos
2                      MAC0422 - Sistemas Operacionais
3                                                  NaN
Name: 0, dtype: object
req[i][0]:  Curso: 45052 Bacharelado em Ciência da Computação (diurno) - Período ideal: 7
4
Req[i]:  0    Curso: 45052 Bacharelado em Ciência da Computa...
1                      MAC0338 - Análise de Algoritmos
2                      MAC0422 - Sistemas Operacionais
3                                                  NaN
Name: 1, dtype: object
req[i][0]:  Curso: 45052 Bacharelado em Ciência da Computação (diurno) - Período ideal: 7
4
Req[i]:  0    Curso: 45052 Bacharelado em Ciência da Computa...
1                                            Requisito
2                                            Requisito
3                                                  NaN
Name: 2, dtype: object
req[i][0]:  Curso: 45052 Bacharelado em Ciência da Computaçã

...

Req[i]:  0    Curso: 45051 Bacharelado em Ciência da Computa...
1              MAC0242 - Laboratório de Programação II
2                                                  NaN
3    Curso: 45052 Bacharelado em Ciência da Computa...
4              MAC0242 - Laboratório de Programação II
5                                                  NaN
Name: 0, dtype: object
req[i][0]:  Curso: 45051 Bacharelado em Ciência da Computação (diurno) - Período ideal: 8
6
Req[i]:  0    Curso: 45051 Bacharelado em Ciência da Computa...
1              MAC0242 - Laboratório de Programação II
2                                                  NaN
3    Curso: 45052 Bacharelado em Ciência da Computa...
4              MAC0242 - Laboratório de Programação II
5                                                  NaN
Name: 1, dtype: object
req[i][0]:  Curso: 45051 Bacharelado em Ciência da Computação (diurno) - Período ideal: 8
6
Req[i]:  0    Curso: 45051 Bacharelado em Ciência da Computa...
1                           

Req[i]:  0    Curso: 45052 Bacharelado em Ciência da Computa...
1         MAC0121 - Algoritmos e Estruturas de Dados I
2                                                  NaN
Name: 0, dtype: object
req[i][0]:  Curso: 45052 Bacharelado em Ciência da Computação (diurno) - Período ideal: 8
3
Req[i]:  0    Curso: 45052 Bacharelado em Ciência da Computa...
1         MAC0121 - Algoritmos e Estruturas de Dados I
2                                                  NaN
Name: 1, dtype: object
req[i][0]:  Curso: 45052 Bacharelado em Ciência da Computação (diurno) - Período ideal: 8
3
Req[i]:  0    Curso: 45052 Bacharelado em Ciência da Computa...
1                                            Requisito
2                                                  NaN
Name: 2, dtype: object
req[i][0]:  Curso: 45052 Bacharelado em Ciência da Computação (diurno) - Período ideal: 8
3
Req[i]:  0    Curso: 45052 Bacharelado em Ciência da Computa...
1                                                  NaN
2             

Req[i]:  0     Curso: 45042 Matemática Aplicada - Bacharelado...
1            MAC0436 - Tópicos de Matemática Discreta I
2                                                   NaN
3     Curso: 45052 Bacharelado em Ciência da Computa...
4            MAC0436 - Tópicos de Matemática Discreta I
5                                                   NaN
6     Curso: 45061 Estatística Bacharelado (diurno) ...
7            MAC0436 - Tópicos de Matemática Discreta I
8                                                   NaN
9     Curso: 45062 Estatística Bacharelado (diurno) ...
10           MAC0436 - Tópicos de Matemática Discreta I
11                                                  NaN
12    Curso: 45070 Bacharelado em Matemática Aplicad...
13           MAC0436 - Tópicos de Matemática Discreta I
14                                                  NaN
Name: 0, dtype: object
req[i][0]:  Curso: 45042 Matemática Aplicada - Bacharelado (diurno) - Período ideal: 8
15
entrou
entrou
Req[i]:  0     Curso: 45

Req[i]:  0    Curso: 45042 Matemática Aplicada - Bacharelado...
1    MAC0775 - Métodos Probabilísticos em Combinató...
2                                                  NaN
3    Curso: 45052 Bacharelado em Ciência da Computa...
4    MAC0775 - Métodos Probabilísticos em Combinató...
5                                                  NaN
6    Curso: 45070 Bacharelado em Matemática Aplicad...
7    MAC0775 - Métodos Probabilísticos em Combinató...
8                                                  NaN
Name: 0, dtype: object
req[i][0]:  Curso: 45042 Matemática Aplicada - Bacharelado (diurno) - Período ideal: 8
9
entrou
entrou
Req[i]:  0    Curso: 45042 Matemática Aplicada - Bacharelado...
1    MAC0775 - Métodos Probabilísticos em Combinató...
2                                                  NaN
3    Curso: 45052 Bacharelado em Ciência da Computa...
4    MAC0775 - Métodos Probabilísticos em Combinató...
5                                                  NaN
6    Curso: 45070 Bacharelado e

Req[i]:  0      Curso: 45031 Matemática Bacharelado (diurno) -...
1                            MAT0222 - Álgebra Linear II
2                      MAT0226 - Equações Diferenciais I
3                                                     ou
4           MAT0205 - Cálculo Diferencial e Integral III
                             ...                        
122                                                   ou
123         MAT3120 - Cálculo Diferencial e Integral III
124                             MAT3211 - Álgebra Linear
125          MAT3220 - Cálculo Diferencial e Integral IV
126                                                  NaN
Name: 0, Length: 127, dtype: object
req[i][0]:  Curso: 45031 Matemática Bacharelado (diurno) - Período ideal: 6
127
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
entrou
Req

In [8]:
print(codigos)

{'MAC0101': [], 'MAC0105': [], 'MAC0110': [], 'MAC0329': [], 'MAT0112': [], 'MAT2453': [], 'MAC0121': ['MAC0110'], 'MAC0216': ['MAC0110'], 'MAC0239': ['MAC0110'], 'MAE0119': [], 'MAT0122': ['MAT0112'], 'MAT2454': ['MAT2453'], 'MAC0102': ['MAC0121'], 'MAC0209': ['MAC0110'], 'MAC0210': ['MAC0110', 'MAT0122'], 'MAC0323': ['MAC0121'], 'MAT0236': ['MAT2454'], 'MAC0316': ['MAC0121'], 'MAC0338': ['MAC0323'], 'MAC0350': ['MAC0316'], 'MAC0422': ['MAC0121'], 'FLC0474': [], 'MAC0499': ['MAC0338', 'MAC0422'], 'GMG0630': [], 'ACH2023': [], 'ACH2033': [], 'MAC0213': [], 'MAC0214': [], 'MAC0215': [], 'CJE0642': [], 'IOF0115': ['MAC0110'], 'MAC0327': ['MAC0121'], 'MAC0337': ['MAC0121'], 'MAC0453': [], 'MAE0217': ['MAE0119'], 'MAE0228': ['MAE0212'], 'MAE0399': ['MAE0119'], 'MAP2001': [], 'MAT0206': ['MAT2454'], 'MAT0222': ['MAT0122'], 'MAT0223': ['MAT0138'], 'MAT0264': ['MAC0105'], 'MAC0218': ['MAC0216'], 'MAC0219': ['MAC0121', 'MAC0216'], 'MAC0242': ['MAC0121'], 'MAC0300': ['MAT0122'], 'MAC0341': [], 